In [1]:
import os
os.chdir("../")
!pwd

/home/aditya/MLOPS/E2E-Data-Science-Project


In [2]:
from pathlib import Path
from pydantic import BaseModel

class ModelTrainerConfig(BaseModel):
    ## from config
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    ## from params
    alpha: float  
    l1_ratio: float
    ## from schema 
    target_column: str

In [3]:
from src.DataScienceWorkflow.constants import *
from src.DataScienceWorkflow.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:

        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_model_trainer_config = ModelTrainerConfig(
            
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha =  params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name,
        )

        return data_model_trainer_config

In [4]:
cm = ConfigurationManager()
data_model_trainer_config = cm.get_model_trainer_config()

[2025-03-29 19:38:48,966: INFO: common : Yaml File: config/config.yaml loaded successfully]
[2025-03-29 19:38:48,967: INFO: common : Yaml File: params.yaml loaded successfully]
[2025-03-29 19:38:48,969: INFO: common : Yaml File: schema.yaml loaded successfully]
[2025-03-29 19:38:48,969: INFO: common : created directory at: artifacts]
[2025-03-29 19:38:48,970: INFO: common : created directory at: artifacts/model_trainer]


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from src.DataScienceWorkflow import logger
from sklearn.linear_model import ElasticNet
import joblib

# perform all EDA here
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config
    
    def train(self):

        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        x_train = train_data.drop([self.config.target_column],axis=1)
        x_test = test_data.drop([self.config.target_column],axis = 1)

        y_train = train_data[self.config.target_column]
        y_test = test_data[self.config.target_column]

        lr = ElasticNet(alpha=self.config.alpha,
                        l1_ratio=self.config.l1_ratio,
                        random_state = 42
                        )
        
        lr.fit(x_train,y_train)

        joblib.dump(lr, os.path.join(self.config.root_dir,self.config.model_name))


In [ ]:
ml = ModelTrainer(data_model_trainer_config)
ml.train()